In [1]:
import json

with open('pa-data-filters.json', 'r') as f:
    data = json.load(f)
    
import openpyxl
from os import listdir
from os.path import isfile, join
mypath = '../vpscrape/getvirons/xlfiles/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

from plot_util import *

def letters(s):
    valids = []
    for character in s:
        if character.isalpha():
            valids.append(character)
    return ''.join(valids)

In [9]:
data['all_pas_closest'] = {}
data['close_aas'] = {}
for f in onlyfiles:
    if 'full' not in f or '.xlsx' not in f: continue
    xldir = mypath + f
    wb = openpyxl.load_workbook(xldir)
    v = f.split('_', 1)[-1].split('.')[0]
    dclosest_aas = []
    for sheetname in wb.sheetnames:
        sh = wb[sheetname]
        vitems = []
        for row in sh.iter_rows():
                vitems.append([cell.value for cell in row])
        aas = []
        cnt = 0
        ds_aas = {}
        for i in range(len(vitems[0])):
            for j in range(2, len(vitems)):
                if vitems[j][i] is not None and '.' in str(vitems[j][i]):
                    d = vitems[j][i]
                    ds_aas[d] = vitems[j][i:i+4]
                    if d < 5:
                        aas.append(letters(vitems[j][i+2]))
                        if vitems[j][i+3] != 'N/A': aas.append(letters(vitems[j][i+3]))
        closest_aa = letters(ds_aas[min(ds_aas.keys())][2])
        other_aa = letters(ds_aas[min(ds_aas.keys())][3])
        dclosest_aas.append([closest_aa, other_aa])
    data['all_pas_closest'][v] = dclosest_aas

In [11]:
#was pa-data-close.json
with open('pa-data-all-pas.json', 'w') as f:
    json_str = json.dumps(data, indent=4)
    f.write(json_str)